In [ ]:
from simple_dmet import *

if __name__ == "__main__":

    # calculating only one fragment
    fragments = [[[0,1], [2,3], [4,5], [6,7], [8,9]]] 

    def make_Hring(l=1.):
        r = l/(2 * np.sin(np.pi / 10))
        atmlst = []
        for i in range(10):
            atmlst.append(['H', (r*np.cos(2. * np.pi/10*i), r*np.sin(2. * np.pi/10*i), 0)])
        mol = gto.Mole()
        mol.atom = atmlst
        mol.basis = 'sto-6g'
        mol.build()
        return mol

    mol = make_Hring(1.4)
    mf = hf.RHF(mol); mf.kernel()
    dmet_outer = DMET_Outer(mf, fragments)
    dmet_outer.chempot_cycle(method='fci')
    etot, nelec, dmx = dmet_outer.inner_kernel(method='fci')

    print(dmx)
    # gradient test
    def f(u):
            
        umat, mo_energy, mo_coeff, mo_occ, dm_mf = dmet_outer.corr_info(u)
        loss = dmloss(dm_mf, dmx, dmet_outer.fragments)
        
        return loss

    def fprime(u):
        grad = np.zeros_like(u)
        umat, mo_energy, mo_coeff, mo_occ, dm_mf = dmet_outer.corr_info(u)
        gradmat = dmloss_grad(dm_mf, dmx, mo_energy, mo_coeff, mo_occ, dmet_outer.fragments)
        gradmat =  (gradmat + gradmat.T) / 2.
        idx = 0
        k = 0

        for sym_frag in dmet_outer.fragments:
            frag0 = sym_frag[0]
            idxr = idx + len(frag0)**2
            grad[idx:idxr] = gradmat[np.ix_(frag0, frag0)].reshape(-1) * dmet_outer.sym_factor[k]
            idx = idxr

            k += 1
        return grad

    def get_num_grad(f, u0, du):
        a = []
        for i in range(len(u0)):
            ul = u0.copy(); ul[i] -= du
            ur = u0.copy(); ur[i] += du
            fl = f(ul); fr = f(ur)
            a.append((fr-fl)/(2.*du))

        return np.array(a)

    u0 = np.array([4E-4, -1E-4, -1E-4, -2E-4])
    num_grad = get_num_grad(f, u0, 1E-6)

    anal_grad = fprime(u0)
    print(num_grad)
    print(anal_grad)